# Coordinate Systems: Transformations and Projections

This notebook covers coordinate system conversions and map projections commonly used in tracking and navigation. We explore:

1. **Geodetic Coordinates** - Latitude, longitude, altitude (WGS84)
2. **ECEF Coordinates** - Earth-Centered Earth-Fixed Cartesian system
3. **Local Frames** - ENU (East-North-Up) and NED (North-East-Down)
4. **Rotation Representations** - Euler angles, quaternions, rotation matrices
5. **Map Projections** - UTM, Mercator, stereographic

## Prerequisites

```bash
pip install nrl-tracker matplotlib numpy
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytcl.coordinate_systems import (
    # Geodetic conversions
    geodetic2ecef, ecef2geodetic,
    ecef2enu, enu2ecef, ecef2ned, ned2ecef,
    enu2ned, ned2enu,
    # Rotation operations
    rotx, roty, rotz,
    euler2rotmat, rotmat2euler,
    euler2quat, quat2euler,
    quat_multiply, quat_rotate, slerp,
    axisangle2rotmat, rotmat2axisangle,
    # Projections
    geodetic2utm, utm2geodetic,
    mercator, mercator_inverse,
    stereographic, polar_stereographic,
    lambert_conformal_conic,
)
from pytcl.core.constants import WGS84

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Geodetic and ECEF Coordinates

### Geodetic Coordinates (LLA)

Geodetic coordinates describe position using:
- **Latitude (φ)**: Angle from equatorial plane (-90° to +90°)
- **Longitude (λ)**: Angle from prime meridian (-180° to +180°)
- **Altitude (h)**: Height above reference ellipsoid

### ECEF Coordinates

Earth-Centered Earth-Fixed (ECEF) is a Cartesian coordinate system:
- **Origin**: Earth's center of mass
- **X-axis**: Points to (0°N, 0°E) - equator/prime meridian intersection
- **Y-axis**: Points to (0°N, 90°E) - equator/90° east
- **Z-axis**: Points to North Pole (90°N)

### Conversion Formulas

The WGS84 ellipsoid is defined by:
- Semi-major axis: a = 6,378,137 m
- Flattening: f = 1/298.257223563

In [ ]:
# Define some example locations
locations = {
    'Washington DC': (38.9072, -77.0369, 0),      # lat, lon (deg), alt (m)
    'London': (51.5074, -0.1278, 0),
    'Tokyo': (35.6762, 139.6503, 0),
    'Sydney': (-33.8688, 151.2093, 0),
    'North Pole': (90.0, 0.0, 0),
    'Equator/Prime': (0.0, 0.0, 0),
}

print("Location Coordinates")
print("=" * 80)
print(f"{'Location':15s} | {'Lat (°)':>10s} | {'Lon (°)':>10s} | "
      f"{'X (km)':>10s} | {'Y (km)':>10s} | {'Z (km)':>10s}")
print("-" * 80)

ecef_coords = {}
for name, (lat_deg, lon_deg, alt) in locations.items():
    lat = np.radians(lat_deg)
    lon = np.radians(lon_deg)
    
    ecef = geodetic2ecef(lat, lon, alt)
    ecef_coords[name] = ecef
    
    print(f"{name:15s} | {lat_deg:10.4f} | {lon_deg:10.4f} | "
          f"{ecef[0]/1e3:10.1f} | {ecef[1]/1e3:10.1f} | {ecef[2]/1e3:10.1f}")

In [ ]:
# Visualize locations in 3D ECEF
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Draw Earth ellipsoid (simplified as sphere)
u = np.linspace(0, 2 * np.pi, 50)
v = np.linspace(0, np.pi, 30)
R = WGS84.a / 1e6  # Scale to millions of meters

x_sphere = R * np.outer(np.cos(u), np.sin(v))
y_sphere = R * np.outer(np.sin(u), np.sin(v))
z_sphere = R * np.outer(np.ones(np.size(u)), np.cos(v)) * (1 - WGS84.f)

ax.plot_surface(x_sphere, y_sphere, z_sphere, alpha=0.3, color='lightblue')

# Plot locations
colors = plt.cm.tab10(np.linspace(0, 1, len(locations)))
for (name, ecef), color in zip(ecef_coords.items(), colors):
    ax.scatter(ecef[0]/1e6, ecef[1]/1e6, ecef[2]/1e6, 
               s=100, c=[color], label=name, depthshade=False)

# Draw axes
axis_length = 8
ax.quiver(0, 0, 0, axis_length, 0, 0, color='red', arrow_length_ratio=0.1)
ax.quiver(0, 0, 0, 0, axis_length, 0, color='green', arrow_length_ratio=0.1)
ax.quiver(0, 0, 0, 0, 0, axis_length, color='blue', arrow_length_ratio=0.1)
ax.text(axis_length, 0, 0, 'X', fontsize=12, color='red')
ax.text(0, axis_length, 0, 'Y', fontsize=12, color='green')
ax.text(0, 0, axis_length, 'Z (North)', fontsize=12, color='blue')

ax.set_xlabel('X (million m)')
ax.set_ylabel('Y (million m)')
ax.set_zlabel('Z (million m)')
ax.set_title('ECEF Coordinate System')
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Verify round-trip conversion accuracy
print("Round-trip Conversion Accuracy (ECEF → Geodetic → ECEF)")
print("=" * 60)

for name, ecef_orig in ecef_coords.items():
    # ECEF to geodetic
    lat, lon, alt = ecef2geodetic(ecef_orig)
    
    # Back to ECEF
    ecef_back = geodetic2ecef(lat, lon, alt)
    
    # Compute error
    error = np.linalg.norm(ecef_orig - ecef_back)
    
    print(f"{name:15s}: Round-trip error = {error:.6e} m")

## 2. Local Tangent Plane Frames: ENU and NED

For local operations near a reference point, we use tangent plane coordinate systems:

### ENU (East-North-Up)
- **East**: Points east (increasing longitude)
- **North**: Points north (increasing latitude)  
- **Up**: Points away from Earth center (opposite gravity)

### NED (North-East-Down)
- **North**: Points north
- **East**: Points east
- **Down**: Points toward Earth center (with gravity)

NED is common in aerospace (aircraft body frame), while ENU is common in robotics.

In [ ]:
# Reference point: Washington DC airport
ref_lat = np.radians(38.9)
ref_lon = np.radians(-77.0)
ref_alt = 0.0

print(f"Reference Point: {np.degrees(ref_lat):.2f}°N, {np.degrees(ref_lon):.2f}°W")
print("\nSimulated aircraft positions relative to reference:")

# Simulate aircraft at various positions
aircraft_offsets = [
    ('Aircraft 1', 1000, 2000, 500),     # 1km E, 2km N, 500m Up
    ('Aircraft 2', -500, 3000, 1000),    # 500m W, 3km N, 1km Up
    ('Aircraft 3', 2000, -1000, 2000),   # 2km E, 1km S, 2km Up
]

print(f"{'Aircraft':12s} | {'East (m)':>10s} | {'North (m)':>10s} | {'Up (m)':>10s}")
print("-" * 55)

ecef_aircraft = []
enu_aircraft = []

for name, east, north, up in aircraft_offsets:
    print(f"{name:12s} | {east:10.0f} | {north:10.0f} | {up:10.0f}")
    
    # Convert ENU offset to ECEF
    enu = np.array([east, north, up])
    ecef = enu2ecef(enu, ref_lat, ref_lon)
    
    enu_aircraft.append(enu)
    ecef_aircraft.append(ecef)

In [ ]:
# Convert same positions to NED
print("\nComparison: ENU vs NED frames")
print("=" * 80)
print(f"{'Aircraft':12s} | {'E (m)':>8s} {'N (m)':>8s} {'U (m)':>8s} | "
      f"{'N (m)':>8s} {'E (m)':>8s} {'D (m)':>8s}")
print(f"{'':12s} | {'--- ENU ---':^26s} | {'--- NED ---':^26s}")
print("-" * 80)

for i, (name, _, _, _) in enumerate(aircraft_offsets):
    enu = enu_aircraft[i]
    ned = enu2ned(enu)
    
    print(f"{name:12s} | {enu[0]:8.0f} {enu[1]:8.0f} {enu[2]:8.0f} | "
          f"{ned[0]:8.0f} {ned[1]:8.0f} {ned[2]:8.0f}")

print("\nNote: NED swaps E↔N and negates Up to get Down")

In [ ]:
# Visualize ENU and NED frames
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ENU plot
ax = axes[0]
for i, (name, _, _, _) in enumerate(aircraft_offsets):
    enu = enu_aircraft[i]
    ax.scatter(enu[0], enu[1], s=100, label=f"{name} (Up={enu[2]:.0f}m)")
    ax.annotate(name, (enu[0], enu[1]), xytext=(5, 5), textcoords='offset points')

ax.scatter(0, 0, s=200, c='red', marker='*', label='Reference', zorder=5)
ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
ax.set_xlabel('East (m)')
ax.set_ylabel('North (m)')
ax.set_title('ENU Frame (looking down)')
ax.legend()
ax.axis('equal')
ax.grid(True)

# NED plot  
ax = axes[1]
for i, (name, _, _, _) in enumerate(aircraft_offsets):
    enu = enu_aircraft[i]
    ned = enu2ned(enu)
    ax.scatter(ned[1], ned[0], s=100, label=f"{name} (Down={ned[2]:.0f}m)")
    ax.annotate(name, (ned[1], ned[0]), xytext=(5, 5), textcoords='offset points')

ax.scatter(0, 0, s=200, c='red', marker='*', label='Reference', zorder=5)
ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
ax.set_xlabel('East (m)')
ax.set_ylabel('North (m)')
ax.set_title('NED Frame (looking down, but X=North, Y=East)')
ax.legend()
ax.axis('equal')
ax.grid(True)

plt.tight_layout()
plt.show()

## 3. Rotation Representations

Rotations in 3D can be represented several ways:

| Representation | Parameters | Singularities | Use Case |
|----------------|-----------|---------------|----------|
| Euler Angles | 3 | Gimbal lock | Human intuition |
| Rotation Matrix | 9 (6 DOF) | None | Direct application |
| Quaternion | 4 (3 DOF) | None | Interpolation, composition |
| Axis-Angle | 3 | Small angles | Visualization |

### Euler Angles (ZYX Convention)

The aerospace convention uses:
- **Yaw (ψ)**: Rotation about Z-axis (heading)
- **Pitch (θ)**: Rotation about Y-axis (nose up/down)
- **Roll (φ)**: Rotation about X-axis (bank)

In [ ]:
# Demonstrate rotation representations
yaw = np.radians(45)    # 45° heading
pitch = np.radians(15)  # 15° nose up
roll = np.radians(10)   # 10° bank right

angles = np.array([yaw, pitch, roll])

print("Input Euler angles (ZYX convention):")
print(f"  Yaw:   {np.degrees(yaw):6.1f}°")
print(f"  Pitch: {np.degrees(pitch):6.1f}°")
print(f"  Roll:  {np.degrees(roll):6.1f}°")

# Convert to rotation matrix
R = euler2rotmat(angles, 'ZYX')
print(f"\nRotation Matrix:\n{R}")

# Convert to quaternion
q = euler2quat(angles, 'ZYX')
print(f"\nQuaternion [w, x, y, z]: {q}")

# Convert back to verify
angles_back = rotmat2euler(R, 'ZYX')
print(f"\nRecovered Euler angles:")
print(f"  Yaw:   {np.degrees(angles_back[0]):6.1f}°")
print(f"  Pitch: {np.degrees(angles_back[1]):6.1f}°")
print(f"  Roll:  {np.degrees(angles_back[2]):6.1f}°")

In [ ]:
# Visualize rotation effect on aircraft axes
fig = plt.figure(figsize=(12, 5))

# Define aircraft body axes (before rotation)
body_axes = np.array([
    [1, 0, 0],  # Forward (nose)
    [0, 1, 0],  # Right wing
    [0, 0, 1],  # Down
]).T

# Apply rotation
rotated_axes = R @ body_axes

# Plot before rotation
ax1 = fig.add_subplot(121, projection='3d')
colors = ['red', 'green', 'blue']
labels = ['Forward (X)', 'Right (Y)', 'Down (Z)']

for i, (color, label) in enumerate(zip(colors, labels)):
    ax1.quiver(0, 0, 0, body_axes[0, i], body_axes[1, i], body_axes[2, i],
               color=color, arrow_length_ratio=0.1, label=label)

ax1.set_xlim([-1, 1])
ax1.set_ylim([-1, 1])
ax1.set_zlim([-1, 1])
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Body Axes (No Rotation)')
ax1.legend()

# Plot after rotation
ax2 = fig.add_subplot(122, projection='3d')

for i, (color, label) in enumerate(zip(colors, labels)):
    ax2.quiver(0, 0, 0, rotated_axes[0, i], rotated_axes[1, i], rotated_axes[2, i],
               color=color, arrow_length_ratio=0.1, label=label)

ax2.set_xlim([-1, 1])
ax2.set_ylim([-1, 1])
ax2.set_zlim([-1, 1])
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.set_title(f'Rotated (Yaw={np.degrees(yaw):.0f}°, Pitch={np.degrees(pitch):.0f}°, Roll={np.degrees(roll):.0f}°)')
ax2.legend()

plt.tight_layout()
plt.show()

## 4. Quaternion Operations

Quaternions are excellent for:
- **Composition**: Multiplying quaternions combines rotations
- **Interpolation**: SLERP provides smooth rotation interpolation
- **Numerical stability**: No gimbal lock issues

In [ ]:
# Demonstrate quaternion operations

# Two rotations to compose
q1 = euler2quat(np.radians([30, 0, 0]), 'ZYX')  # 30° yaw
q2 = euler2quat(np.radians([0, 20, 0]), 'ZYX')  # 20° pitch

print("Composing rotations:")
print(f"  q1 (30° yaw):   {q1}")
print(f"  q2 (20° pitch): {q2}")

# Multiply quaternions (composition)
q_composed = quat_multiply(q1, q2)
print(f"  q1 * q2:        {q_composed}")

# Convert result back to Euler
euler_composed = quat2euler(q_composed, 'ZYX')
print(f"\nComposed rotation (Euler):")
print(f"  Yaw: {np.degrees(euler_composed[0]):.1f}°, "
      f"Pitch: {np.degrees(euler_composed[1]):.1f}°, "
      f"Roll: {np.degrees(euler_composed[2]):.1f}°")

In [ ]:
# SLERP interpolation demonstration
q_start = euler2quat(np.radians([0, 0, 0]), 'ZYX')    # No rotation
q_end = euler2quat(np.radians([180, 45, 30]), 'ZYX')  # Complex rotation

# Interpolate
t_values = np.linspace(0, 1, 11)
interpolated = []

print("SLERP Interpolation:")
print(f"{'t':>5s} | {'Yaw (°)':>10s} | {'Pitch (°)':>10s} | {'Roll (°)':>10s}")
print("-" * 45)

for t in t_values:
    q_interp = slerp(q_start, q_end, t)
    euler = quat2euler(q_interp, 'ZYX')
    interpolated.append(np.degrees(euler))
    print(f"{t:5.2f} | {np.degrees(euler[0]):10.2f} | "
          f"{np.degrees(euler[1]):10.2f} | {np.degrees(euler[2]):10.2f}")

interpolated = np.array(interpolated)

In [ ]:
# Visualize SLERP interpolation
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(t_values, interpolated[:, 0], 'b-o', label='Yaw')
ax.plot(t_values, interpolated[:, 1], 'r-s', label='Pitch')
ax.plot(t_values, interpolated[:, 2], 'g-^', label='Roll')

ax.set_xlabel('Interpolation Parameter t')
ax.set_ylabel('Angle (degrees)')
ax.set_title('SLERP Quaternion Interpolation')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

## 5. Map Projections

Map projections convert 3D geodetic coordinates to 2D plane coordinates. Different projections preserve different properties:

| Projection | Preserves | Distorts | Use Case |
|------------|-----------|----------|----------|
| Mercator | Angles (conformal) | Area | Navigation, web maps |
| UTM | Local distances | At zone edges | Military, surveying |
| Lambert | Angles (conformal) | Poles | Mid-latitude regions |
| Stereographic | Angles (conformal) | Edges | Polar regions |

In [ ]:
# UTM projection example
print("UTM Projection Examples")
print("=" * 70)

for name, (lat_deg, lon_deg, _) in locations.items():
    lat = np.radians(lat_deg)
    lon = np.radians(lon_deg)
    
    # Skip poles (UTM not defined there)
    if abs(lat_deg) > 84:
        print(f"{name:15s}: UTM not defined (use UPS for polar regions)")
        continue
    
    utm = geodetic2utm(lat, lon)
    
    print(f"{name:15s}: Zone {utm.zone:2d}{utm.hemisphere}, "
          f"E={utm.easting:10.1f}m, N={utm.northing:11.1f}m, "
          f"Scale={utm.scale:.6f}")

In [ ]:
# Compare projections for a region
# Generate a grid of points around Washington DC
center_lat = np.radians(38.9)
center_lon = np.radians(-77.0)

# Create a 10° x 10° grid
lat_range = np.linspace(center_lat - np.radians(5), center_lat + np.radians(5), 20)
lon_range = np.linspace(center_lon - np.radians(5), center_lon + np.radians(5), 20)

LAT, LON = np.meshgrid(lat_range, lon_range)

# Project using different methods
mercator_x = np.zeros_like(LAT)
mercator_y = np.zeros_like(LAT)
stereo_x = np.zeros_like(LAT)
stereo_y = np.zeros_like(LAT)

for i in range(LAT.shape[0]):
    for j in range(LAT.shape[1]):
        # Mercator
        merc = mercator(LAT[i, j], LON[i, j], center_lon)
        mercator_x[i, j] = merc.x / 1e6
        mercator_y[i, j] = merc.y / 1e6
        
        # Stereographic
        ster = stereographic(LAT[i, j], LON[i, j], center_lat, center_lon)
        stereo_x[i, j] = ster.x / 1e6
        stereo_y[i, j] = ster.y / 1e6

In [ ]:
# Visualize projection comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original lat/lon grid
ax = axes[0]
ax.pcolormesh(np.degrees(LON), np.degrees(LAT), np.zeros_like(LAT), 
              alpha=0.3, shading='auto')
for i in range(LAT.shape[0]):
    ax.plot(np.degrees(LON[i, :]), np.degrees(LAT[i, :]), 'b-', alpha=0.5)
for j in range(LAT.shape[1]):
    ax.plot(np.degrees(LON[:, j]), np.degrees(LAT[:, j]), 'b-', alpha=0.5)
ax.scatter(np.degrees(center_lon), np.degrees(center_lat), c='red', s=100, zorder=5)
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.set_title('Geographic (Lat/Lon)')
ax.set_aspect('equal')

# Mercator
ax = axes[1]
for i in range(LAT.shape[0]):
    ax.plot(mercator_x[i, :], mercator_y[i, :], 'g-', alpha=0.5)
for j in range(LAT.shape[1]):
    ax.plot(mercator_x[:, j], mercator_y[:, j], 'g-', alpha=0.5)
ax.scatter(0, mercator_y[10, 10], c='red', s=100, zorder=5)
ax.set_xlabel('Easting (million m)')
ax.set_ylabel('Northing (million m)')
ax.set_title('Mercator Projection')
ax.set_aspect('equal')

# Stereographic
ax = axes[2]
for i in range(LAT.shape[0]):
    ax.plot(stereo_x[i, :], stereo_y[i, :], 'm-', alpha=0.5)
for j in range(LAT.shape[1]):
    ax.plot(stereo_x[:, j], stereo_y[:, j], 'm-', alpha=0.5)
ax.scatter(0, 0, c='red', s=100, zorder=5)
ax.set_xlabel('Easting (million m)')
ax.set_ylabel('Northing (million m)')
ax.set_title('Stereographic Projection')
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

In [ ]:
# UTM zone visualization
print("UTM Zone Coverage")
print("=" * 50)

# Show UTM zones for various longitudes
longitudes = np.arange(-180, 181, 30)
print(f"{'Longitude':>12s} | {'UTM Zone':>10s} | {'Central Meridian':>18s}")
print("-" * 50)

for lon_deg in longitudes:
    lon = np.radians(lon_deg)
    lat = 0  # Equator
    
    utm_result = geodetic2utm(lat, lon)
    
    from pytcl.coordinate_systems import utm_central_meridian
    central = np.degrees(utm_central_meridian(utm_result.zone))
    
    print(f"{lon_deg:12.0f}° | {utm_result.zone:10d} | {central:18.0f}°")

## 6. Practical Example: Aircraft Tracking

Let's put it all together with a complete aircraft tracking example.

In [ ]:
# Simulate an aircraft trajectory
# Takeoff from Washington DC, fly northeast

# Radar station location
radar_lat = np.radians(38.9)
radar_lon = np.radians(-77.0)
radar_alt = 100.0  # 100m tower

# Generate flight path (geodetic)
n_points = 50
t = np.linspace(0, 1, n_points)

# Start position
start_lat = np.radians(38.95)
start_lon = np.radians(-77.05)

# End position (100km northeast)
end_lat = np.radians(39.7)
end_lon = np.radians(-76.2)

# Interpolate position
flight_lat = start_lat + t * (end_lat - start_lat)
flight_lon = start_lon + t * (end_lon - start_lon)
flight_alt = 1000 + 9000 * np.sin(np.pi * t)  # Climb to 10km, then descend

print(f"Flight from ({np.degrees(start_lat):.2f}°, {np.degrees(start_lon):.2f}°) "
      f"to ({np.degrees(end_lat):.2f}°, {np.degrees(end_lon):.2f}°)")
print(f"Max altitude: {max(flight_alt):.0f} m")

In [ ]:
# Convert to various coordinate systems
flight_ecef = np.array([geodetic2ecef(lat, lon, alt) 
                        for lat, lon, alt in zip(flight_lat, flight_lon, flight_alt)])

flight_enu = np.array([ecef2enu(ecef, radar_lat, radar_lon) 
                       for ecef in flight_ecef])

flight_ned = np.array([ecef2ned(ecef, radar_lat, radar_lon) 
                       for ecef in flight_ecef])

# Compute range and angles from radar
ranges = np.linalg.norm(flight_enu, axis=1)
azimuths = np.degrees(np.arctan2(flight_enu[:, 0], flight_enu[:, 1]))  # From north
elevations = np.degrees(np.arcsin(flight_enu[:, 2] / ranges))

print("\nFlight Path Statistics:")
print(f"  Range: {ranges[0]/1e3:.1f} km to {ranges.max()/1e3:.1f} km")
print(f"  Azimuth: {azimuths[0]:.1f}° to {azimuths[-1]:.1f}°")
print(f"  Elevation: {elevations[0]:.1f}° to {elevations.max():.1f}°")

In [ ]:
# Comprehensive visualization
fig = plt.figure(figsize=(15, 10))

# 3D ENU view
ax1 = fig.add_subplot(221, projection='3d')
ax1.plot(flight_enu[:, 0]/1e3, flight_enu[:, 1]/1e3, flight_enu[:, 2]/1e3, 
         'b-', linewidth=2, label='Flight path')
ax1.scatter([0], [0], [0.1], c='red', s=100, marker='^', label='Radar')
ax1.scatter(flight_enu[0, 0]/1e3, flight_enu[0, 1]/1e3, flight_enu[0, 2]/1e3,
           c='green', s=100, marker='o', label='Takeoff')
ax1.scatter(flight_enu[-1, 0]/1e3, flight_enu[-1, 1]/1e3, flight_enu[-1, 2]/1e3,
           c='orange', s=100, marker='s', label='Landing')
ax1.set_xlabel('East (km)')
ax1.set_ylabel('North (km)')
ax1.set_zlabel('Up (km)')
ax1.set_title('ENU Coordinates from Radar')
ax1.legend()

# 2D overhead view
ax2 = fig.add_subplot(222)
scatter = ax2.scatter(flight_enu[:, 0]/1e3, flight_enu[:, 1]/1e3, 
                      c=flight_alt/1e3, cmap='viridis', s=20)
ax2.scatter([0], [0], c='red', s=100, marker='^', label='Radar')
ax2.plot(flight_enu[:, 0]/1e3, flight_enu[:, 1]/1e3, 'b-', alpha=0.3)
plt.colorbar(scatter, ax=ax2, label='Altitude (km)')
ax2.set_xlabel('East (km)')
ax2.set_ylabel('North (km)')
ax2.set_title('Overhead View')
ax2.legend()
ax2.axis('equal')
ax2.grid(True)

# Range-altitude profile
ax3 = fig.add_subplot(223)
horizontal_range = np.sqrt(flight_enu[:, 0]**2 + flight_enu[:, 1]**2) / 1e3
ax3.fill_between(horizontal_range, 0, flight_alt/1e3, alpha=0.3)
ax3.plot(horizontal_range, flight_alt/1e3, 'b-', linewidth=2)
ax3.axhline(y=0, color='brown', linewidth=3, label='Ground')
ax3.set_xlabel('Horizontal Range (km)')
ax3.set_ylabel('Altitude (km)')
ax3.set_title('Range-Altitude Profile')
ax3.grid(True)

# Radar angles
ax4 = fig.add_subplot(224)
ax4.plot(t * 100, azimuths, 'g-', linewidth=2, label='Azimuth')
ax4.plot(t * 100, elevations, 'r-', linewidth=2, label='Elevation')
ax4.set_xlabel('Flight Progress (%)')
ax4.set_ylabel('Angle (degrees)')
ax4.set_title('Radar Angles')
ax4.legend()
ax4.grid(True)

plt.tight_layout()
plt.show()

## Summary

Key takeaways:

1. **Geodetic coordinates** are intuitive but require conversion for calculations
2. **ECEF** is ideal for global calculations but unintuitive for local work
3. **ENU/NED** local frames are best for tracking near a reference point
4. **Quaternions** avoid gimbal lock and provide smooth interpolation
5. **Map projections** trade off different properties - choose based on use case

## Exercises

1. Implement a coordinate converter that handles all transitions between geodetic, ECEF, ENU, and NED
2. Compare quaternion SLERP to linear interpolation of Euler angles
3. Visualize how projection distortion varies with distance from the projection center
4. Track a satellite in ECEF and display its ground track on a map projection

## References

1. Groves, P. D. (2013). *Principles of GNSS, Inertial, and Multisensor Integrated Navigation Systems*
2. Snyder, J. P. (1987). *Map Projections: A Working Manual*. USGS Professional Paper 1395.
3. Kuipers, J. B. (1999). *Quaternions and Rotation Sequences*.